# Encoding complex data with fuzzy logic 

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tommoral/24-sacl-ai-4-sciences/blob/main/session-3/01-complex-tabular-data.ipynb)

Authors: Thomas Moreau, Mathurin Massias

In this notebook we briefly highlight some features of [`skrub`](https://skrub-data.org), a package to preprocess and handle text data in ML pipelines.

If you have not done so, please install [`skrub`](https://skrub-data.org) by uncommenting the following cell:

In [ ]:
# !pip install -U scikit-learn skrub

We will work with public servant data from the US administration, that you can load with the following:

In [ ]:
import pandas as pd

from pathlib import Path


DATA_FILE = "data/salary_X.parquet"
URL_REPO = "https://github.com/x-datascience-datacamp/datacamp-master/raw/main/06-feature_engineering/"

if Path(DATA_FILE).exists():
    data_file = DATA_FILE
else:
    data_file = f"{URL_REPO}{DATA_FILE}"

# Loading data
X = pd.read_parquet(data_file).drop(columns='Unnamed: 0')
y = pd.read_parquet(data_file.replace("_X", "_y")).drop(columns='Unnamed: 0')

print(f'Number of entries & columns in X: {X.shape}')
print(X.columns, y.columns)

In [ ]:
X

In [ ]:
y

Across the administration, the position titles can have similar levels but are not unique.
Also, in some position name, the grade is actually encoded:  `Social Worker IV`, `Resident Supervisor II`, ...
With this, there is a large number of categories that have some similarities:

In [ ]:
X['employee_position_title'].nunique()

If we can learn a model with only with these categories, we can see that some categories have not been seen at test time.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel())

model = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ['employee_position_title'])),
    HistGradientBoostingRegressor()
)
model.fit(X_train, y_train)
model.score(X_test, y_test)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Fix the behavior of `OrdinalEncoder` to allow the model to predict even in the case where some categories are present in the test set and not in the train one.</li>
    </ul>
    
   *Hint:* you can look at the documentation of the [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) to see how to handle missing categories.
</div>

Solution: `solutions/01-1_encode_unknown_values.py`

### Handeling fuzzy categories with `skrub`

In order to leverage the high cardinality categories with similar texts, [`skrub`](https://skrub-data.org/) provide the `GapEncoder`, which performs some fuzy text matching, based on their similarity scores from n-grams:

In [ ]:
import pandas as pd
from skrub import GapEncoder

# defining data
data = pd.Series([
    "Math, optimization",
    "mathematics",
    "maths, ml",
    "ml.maths",
    "machine learning",
    "physics",
    "phy",
    "statistical physics",
    "computational phys.",
    "comp. maths"
]
)

gap_encoder = GapEncoder(n_components=2, random_state=42)
encoded_data = gap_encoder.fit_transform(data)
print(gap_encoder.get_feature_names_out())

encoded_data['original'] = data
encoded_data = encoded_data.set_index('original')
encoded_data.columns = [0, 1]
print(encoded_data)

To regress the annual salary of each worker based on their title, you can thus transform the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import make_column_transformer

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel())

model = make_pipeline(
    make_column_transformer((GapEncoder(n_components=30), 'employee_position_title')),
    HistGradientBoostingRegressor()
)
model.fit(X_train, y_train).score(X_test, y_test)


In [ ]:
gapencoder = model[0]
gapencoder.named_transformers_['gapencoder'].get_feature_names_out()

### Automatized feature extraction with complex tabular data

The original data contains various columns, with categorical features and dates.
[`skrub`](https://skrub-data.org/) provides a convenient tools to directly vectorize the full tables with reasonable defaults:

In [ ]:
from skrub import TableVectorizer


# defining pipeline
model = make_pipeline(
    TableVectorizer(),
    HistGradientBoostingRegressor()
)

# fitting model
model.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
# retrieving fitted vectorizer
model.named_steps['tablevectorizer']

But the default preprocessing choices depends on the actual classifier which is put after the `TableVectorizer`.
For this, [`skrub`](https://skrub-data.org/) exposes an helper `make_tabular_model` which change the default based on the actual classifier which is chosen.

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Construct a pipeline with `tabular_learner` for a `Ridge` estimator.</li>
    </ul>
</div>

Solution: `solutions/01-2_encode_unknown_values.py`